-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Exercise #3 - Create Fact & Dim Tables

Now that the three years of orders are combined into a single dataset, we can begin the processes of transforming the data.

In the one record, there are actually four sub-datasets:
* The order itself which is the aggregator of the other three datasets.
* The line items of each order which includes the price and quantity of each specific item.
* The sales rep placing the order.
* The customer placing the order - for the sake of simplicity, we will **not** break this dataset out and leave it as part of the order.

What we want to do next, is to extract all that data into their respective datasets (except the customer data). 

In other words, we want to normalize the data, in this case, to reduce data duplication.

This exercise is broken up into 5 steps:
* Exercise 3.A - Create & Use Database
* Exercise 3.B - Load & Cache Batch Orders
* Exercise 3.C - Extract Sales Reps
* Exercise 3.D - Extract Orders
* Exercise 3.E - Extract Line Items

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Setup Exercise #3</h2>

To get started, run the following cell to setup this exercise, declaring exercise-specific variables and functions.

In [0]:
%run ./_includes/Setup-Exercise-03

Variable/Function,Description
username,class+001@databricks.com
,This is the email address that you signed into Databricks with
working_dir,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone
,This is the directory in which all work should be conducted
user_db,dbacademy_class_001_databricks_com_developer_foundations_capstone
,The name of the database you will use for this project.
batch_source_path,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/batch_orders_dirty.delta
,"The location of the combined, raw, batch of orders."
orders_table,orders
,The name of the orders table.


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.A - Create &amp; Use Database</h2>

By using a specific database, we can avoid contention to commonly named tables that may be in use by other users of the workspace.

**In this step you will need to:**
* Create the database identified by the variable **`user_db`**
* Use the database identified by the variable **`user_db`** so that any tables created in this notebook are **NOT** added to the **`default`** database

**Special Notes**
* Do not hard-code the database name - in some scenarios this will result in validation errors.
* For assistence with the SQL command to create a database, see <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-create-database.html" target="_blank">CREATE DATABASE</a> on the Databricks docs website.
* For assistence with the SQL command to use a database, see <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/sql-ref-syntax-ddl-usedb.html" target="_blank">USE DATABASE</a> on the Databricks docs website.

### Implement Exercise #3.A

Implement your solution in the following cell:

In [0]:
%sql 

create schema  IF NOT EXISTS  dbacademy_class_001_databricks_com_developer_foundations_capstone ;


In [0]:
%sql

USE dbacademy_class_001_databricks_com_developer_foundations_capstone

### Reality Check #3.A
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_a()

Points,Test,Result
1,Using DBR 9.1 & Proper Cluster Configuration,
1,Valid Registration ID,
1,The current database is dbacademy_class_001_databricks_com_developer_foundations_capstone,


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.B - Load &amp; Cache Batch Orders</h2>

Next, we need to load the batch orders from the previous exercise and then cache them in preparation to transform the data later in this exercise.

**In this step you will need to:**
* Load the delta dataset we created in the previous exercise, identified by the variable **`batch_source_path`**.
* Using that same dataset, create a temporary view identified by the variable **`batch_temp_view`**.
* Cache the temporary view.

### Implement Exercise #3.B

Implement your solution in the following cell:

In [0]:
# TODO
# Use this cell to complete your solution

delta = spark.read.format("delta").load(batch_source_path)

display(delta)

submitted_at,order_id,customer_id,sales_rep_id,sales_rep_ssn,sales_rep_first_name,sales_rep_last_name,sales_rep_address,sales_rep_city,sales_rep_state,sales_rep_zip,shipping_address_attention,shipping_address_address,shipping_address_city,shipping_address_state,shipping_address_zip,product_id,product_quantity,product_sold_price,ingest_file_name,ingested_at
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,7a41323a-560f-4e34-aba6-995e2325f95e,300,87.50,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,8d809e13-fdc5-4d15-9271-953750f6d592,800,97.23,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,95cbadca-cf90-4b8a-a134-2976f6ba6df8,800,92.37,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,a8fbcfea-4352-4c5a-af8b-c8623258b4f8,200,96.25,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,a990d79b-4957-42fc-8e42-20ceb1fd1259,600,106.95,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,bc93ed89-bb15-4e46-a110-a5878e46ccf6,200,87.50,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,e26839a2-44fd-4003-a06b-faf6a2dff077,100,92.37,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1504263600,0002589b-d84c-467b-a7b1-de4342812f75,2ac6fe34-26c8-4760-945c-f8b35eb12795,09e2ca9b-a241-4f63-a6a1-5bf63aeeb870,446912278,Cayson,Wiggins,607 S Woodridge Drive,Vacaville,CA,95851,Lena May,677 Red Hill Road W,Chicago,IL,61729,e672483e-57a8-434a-bc42-ecf827c8a8d4,100,101.60,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000
1494810000,0003a3e6-a9f0-49ac-a0c5-75e5d5b149e1,992384ad-ba2c-449f-8304-34617cbe1148,9ef74d02-fe6d-42f3-b638-29e67bbfa20e,337-30-1919,Ruby,Sanford,141 Bosie Run N,Richmond,CA,90041,Misael Fuller,214 W Golden Grove Drive,Anaheim,CA,90224,7a41323a-560f-4e34-aba6-995e2325f95e,800,96.94,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/20

In [0]:
view = delta.createOrReplaceTempView(batch_temp_view)


In [0]:
type(view)

Out[17]: NoneType

In [0]:
%sql
cache table batched_orders

In [0]:
%sql

select * 
from batched_orders

submitted_at,order_id,customer_id,sales_rep_id,sales_rep_ssn,sales_rep_first_name,sales_rep_last_name,sales_rep_address,sales_rep_city,sales_rep_state,sales_rep_zip,shipping_address_attention,shipping_address_address,shipping_address_city,shipping_address_state,shipping_address_zip,product_id,product_quantity,product_sold_price,ingest_file_name,ingested_at
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,7a41323a-560f-4e34-aba6-995e2325f95e,300,85.79,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,7b547a10-e804-48e1-ad90-1f946cee659c,600,95.32,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,8d809e13-fdc5-4d15-9271-953750f6d592,500,95.32,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,95cbadca-cf90-4b8a-a134-2976f6ba6df8,400,90.55,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,a8fbcfea-4352-4c5a-af8b-c8623258b4f8,700,94.37,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,a990d79b-4957-42fc-8e42-20ceb1fd1259,900,104.85,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,bc93ed89-bb15-4e46-a110-a5878e46ccf6,500,85.79,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,e672483e-57a8-434a-bc42-ecf827c8a8d4,1000,99.61,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,ec15ba1d-53b6-44b0-8a22-1e498485f1b8,400,85.79,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-caps

### Reality Check #3.B
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_b()

Points,Test,Result
1,The current database is dbacademy_class_001_databricks_com_developer_foundations_capstone,
1,The table batched_orders exists,
1,The table batched_orders is a temp view,
1,The table batched_orders is cached,
1,"Expected 1,175,870 records",


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.C - Extract Sales Reps</h2>

Our batched orders from Exercise #2 contains thousands of orders and with every order, is the name, SSN, address and other information on the sales rep making the order.

We can use this data to create a table of just our sales reps.

If you consider that we have only ~100 sales reps, but thousands of orders, we are going to have a lot of duplicate data in this space.

Also unique to this set of data, is the fact that social security numbers were not always sanitized meaning sometime they were formatted with hyphens and in other cases they were not - this is something we will have to address here.

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* The SSN numbers have errors in them that we want to track - add the **`boolean`** column **`_error_ssn_format`** - for any case where **`sales_rep_ssn`** has a hypen in it, set this value to **`true`** otherwise **`false`**
* Convert various columns from their string representation to the specified type:
  * The column **`sales_rep_ssn`** should be represented as a **`Long`** (Note: You will have to first clean the column by removing extreneous hyphens in some records)
  * The column **`sales_rep_zip`** should be represented as an **`Integer`**
* Remove the columns not directly related to the sales-rep record:
  * Unrelated ID columns: **`submitted_at`**, **`order_id`**, **`customer_id`**
  * Shipping address columns: **`shipping_address_attention`**, **`shipping_address_address`**, **`shipping_address_city`**, **`shipping_address_state`**, **`shipping_address_zip`**
  * Product columns: **`product_id`**, **`product_quantity`**, **`product_sold_price`**
* Because there is one record per product ordered (many products per order), not to mention one sales rep placing many orders (many orders per sales rep), there will be duplicate records for our sales reps. Remove all duplicate records, making sure to exclude **`ingest_file_name`** and **`ingested_at`** from the evaluation of duplicate records
* Load the dataset to the managed delta table **`sales_rep_scd`** (identified by the variable **`sales_reps_table`**)

**Additional Requirements:**<br/>
The schema for the **`sales_rep_scd`** table must be:
* **`sales_rep_id`**:**`string`**
* **`sales_rep_ssn`**:**`long`**
* **`sales_rep_first_name`**:**`string`**
* **`sales_rep_last_name`**:**`string`**
* **`sales_rep_address`**:**`string`**
* **`sales_rep_city`**:**`string`**
* **`sales_rep_state`**:**`string`**
* **`sales_rep_zip`**:**`integer`**
* **`ingest_file_name`**:**`string`**
* **`ingested_at`**:**`timestamp`**
* **`_error_ssn_format`**:**`boolean`**

### Implement Exercise #3.C

Implement your solution in the following cell:

In [0]:
batched_orders = spark.read.format("delta").table(batch_temp_view)

In [0]:
from pyspark.sql.functions import col, when , regexp_replace
from pyspark.sql.types import NumericType, IntegerType, LongType

subset=["sales_rep_id","sales_rep_ssn","sales_rep_first_name","sales_rep_last_name","sales_rep_address","sales_rep_city","sales_rep_state","sales_rep_zip"]

dataset = batched_orders\
.withColumn("_error_ssn_format", when(col("sales_rep_ssn").contains('-'), True ).otherwise(False))\
.withColumn("sales_rep_ssn", regexp_replace(col("sales_rep_ssn"),"-",""))\
.withColumn("sales_rep_ssn", col("sales_rep_ssn").cast(LongType()))\
.withColumn("sales_rep_zip", col("sales_rep_zip").cast(IntegerType()))\
.drop("submitted_at","order_id","customer_id","shipping_address_attention","shipping_address_address","shipping_address_city","shipping_address_state"
      ,"shipping_address_zip","product_id","product_quantity","product_sold_price")\
.drop_duplicates(subset=subset)
#.groupBy("sales_rep_id","sales_rep_ssn","sales_rep_first_name","sales_rep_last_name","sales_rep_address","sales_rep_city","sales_rep_state","sales_rep_zip","ingest_file_name")\



dataset.write.format("delta").mode("overwrite").saveAsTable(sales_reps_table)



#Unrelated ID columns: submitted_at, order_id, customer_id
#Shipping address columns: , , , , 
#Product columns: , , 


In [0]:
dataset.printSchema()

root
-- sales_rep_id: string (nullable = true)
-- sales_rep_ssn: long (nullable = true)
-- sales_rep_first_name: string (nullable = true)
-- sales_rep_last_name: string (nullable = true)
-- sales_rep_address: string (nullable = true)
-- sales_rep_city: string (nullable = true)
-- sales_rep_state: string (nullable = true)
-- sales_rep_zip: integer (nullable = true)
-- ingest_file_name: string (nullable = true)
-- ingested_at: timestamp (nullable = true)
-- _error_ssn_format: boolean (nullable = false)

In [0]:
batched_orders.filter(col("sales_rep_ssn").contains("-")).show()

+------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+-------------------+--------------+---------------+-------------+--------------------------+------------------------+---------------------+----------------------+--------------------+--------------------+----------------+------------------+--------------------+--------------------+
submitted_at| order_id| customer_id| sales_rep_id|sales_rep_ssn|sales_rep_first_name|sales_rep_last_name| sales_rep_address|sales_rep_city|sales_rep_state|sales_rep_zip|shipping_address_attention|shipping_address_address|shipping_address_city|shipping_address_state|shipping_address_zip| product_id|product_quantity|product_sold_price| ingest_file_name| ingested_at|
+------------+--------------------+--------------------+--------------------+-------------+--------------------+-------------------+-------------------+--------------+---------------+-------------+--------------------------+------------------------+---------------------+----------------------+--------------------+--------------------+----------------+------------------+--------------------+--------------------+
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|7a41323a-560f-4e3...| 300| 85.79|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|7b547a10-e804-48e...| 600| 95.32|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|8d809e13-fdc5-4d1...| 500| 95.32|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|95cbadca-cf90-4b8...| 400| 90.55|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|a8fbcfea-4352-4c5...| 700| 94.37|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|a990d79b-4957-42f...| 900| 104.85|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|bc93ed89-bb15-4e4...| 500| 85.79|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|e672483e-57a8-434...| 1000| 99.61|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1522108800|a661db95-dbe3-470...|251be31f-6593-466...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Kamila Glenn| 979 Carroll Court| Tulsa| OK| 73533|ec15ba1d-53b6-44b...| 400| 85.79|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1523678400|a66b71d9-c5bc-4b9...|f84bea75-f315-41b...|9905fe2c-5438-4ce...| 974-93-2513| Rayne| Jacobson|579 Wayside Place W| Philadelphia| PA| 16652| Lillie Brewer| 714 Nostalgia Ter...| Buffalo| NY| 12975|ec15ba1d-53b6-44b...| 200| 85.79|dbfs:/dbacademy/c...|2022-03-01 15:42:...|
 1539669600|a673b1a4-1129

### Reality Check #3.C
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_c()

Points,Test,Result
1,The current database is dbacademy_class_001_databricks_com_developer_foundations_capstone,
1,The table sales_reps exists,
1,The table sales_reps is a managed table,
1,Using the Delta file format,
1,Schema is valid,
1,Expected 93 records,
1,Expected _error_ssn_format record count to be 17,


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.D - Extract Orders</h2>

Our batched orders from Exercise 02 contains one line per product meaning there are multiple records per order.

The goal of this step is to extract just the order details (excluding the sales rep and line items)

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* Convert various columns from their string representation to the specified type:
  * The column **`submitted_at`** is a "unix epoch" (number of seconds since 1970-01-01 00:00:00 UTC) and should be represented as a **`Timestamp`**
  * The column **`shipping_address_zip`** should be represented as an **`Integer`**
* Remove the columns not directly related to the order record:
  * Sales reps columns: **`sales_rep_ssn`**, **`sales_rep_first_name`**, **`sales_rep_last_name`**, **`sales_rep_address`**, **`sales_rep_city`**, **`sales_rep_state`**, **`sales_rep_zip`**
  * Product columns: **`product_id`**, **`product_quantity`**, **`product_sold_price`**
* Because there is one record per product ordered (many products per order), there will be duplicate records for each order. Remove all duplicate records, making sure to exclude **`ingest_file_name`** and **`ingested_at`** from the evaluation of duplicate records
* Add the column **`submitted_yyyy_mm`** which is a **`string`** derived from **`submitted_at`** and is formatted as "**yyyy-MM**".
* Load the dataset to the managed delta table **`orders`** (identified by the variable **`orders_table`**)
  * In thise case, the data must also be partitioned by **`submitted_yyyy_mm`**

**Additional Requirements:**
* The schema for the **`orders`** table must be:
  * **`submitted_at:timestamp`**
  * **`submitted_yyyy_mm`** using the format "**yyyy-MM**"
  * **`order_id:string`**
  * **`customer_id:string`**
  * **`sales_rep_id:string`**
  * **`shipping_address_attention:string`**
  * **`shipping_address_address:string`**
  * **`shipping_address_city:string`**
  * **`shipping_address_state:string`**
  * **`shipping_address_zip:integer`**
  * **`ingest_file_name:string`**
  * **`ingested_at:timestamp`**

### Implement Exercise #3.D

Implement your solution in the following cell:

In [0]:
batch_temp_view

Out[24]: 'batched_orders'

In [0]:
display(batched_orders)
batched_orders.printSchema

submitted_at,order_id,customer_id,sales_rep_id,sales_rep_ssn,sales_rep_first_name,sales_rep_last_name,sales_rep_address,sales_rep_city,sales_rep_state,sales_rep_zip,shipping_address_attention,shipping_address_address,shipping_address_city,shipping_address_state,shipping_address_zip,product_id,product_quantity,product_sold_price,ingest_file_name,ingested_at
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,7a41323a-560f-4e34-aba6-995e2325f95e,300,85.79,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,7b547a10-e804-48e1-ad90-1f946cee659c,600,95.32,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,8d809e13-fdc5-4d15-9271-953750f6d592,500,95.32,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,95cbadca-cf90-4b8a-a134-2976f6ba6df8,400,90.55,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,a8fbcfea-4352-4c5a-af8b-c8623258b4f8,700,94.37,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,a990d79b-4957-42fc-8e42-20ceb1fd1259,900,104.85,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,bc93ed89-bb15-4e46-a110-a5878e46ccf6,500,85.79,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,e672483e-57a8-434a-bc42-ecf827c8a8d4,1000,99.61,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000
1522108800,a661db95-dbe3-4709-9099-adf9c001ea44,251be31f-6593-4668-8daa-4a0339185298,9905fe2c-5438-4cef-ab4b-666a027f7585,974-93-2513,Rayne,Jacobson,579 Wayside Place W,Philadelphia,PA,16652,Kamila Glenn,979 Carroll Court,Tulsa,OK,73533,ec15ba1d-53b6-44b0-8a22-1e498485f1b8,400,85.79,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-caps

Out[25]: <bound method DataFrame.printSchema of DataFrame[submitted_at: string, order_id: string, customer_id: string, sales_rep_id: string, sales_rep_ssn: string, sales_rep_first_name: string, sales_rep_last_name: string, sales_rep_address: string, sales_rep_city: string, sales_rep_state: string, sales_rep_zip: string, shipping_address_attention: string, shipping_address_address: string, shipping_address_city: string, shipping_address_state: string, shipping_address_zip: string, product_id: string, product_quantity: string, product_sold_price: string, ingest_file_name: string, ingested_at: timestamp]>

In [0]:
# TODO
# Use this cell to complete your solution

from pyspark.sql.types import TimestampType, IntegerType, StringType, StructType, StructField
from pyspark.sql.functions import to_timestamp, from_unixtime

subset_2=["submitted_at","order_id","customer_id","sales_rep_id","shipping_address_attention","shipping_address_address"
          ,"shipping_address_city","shipping_address_state","shipping_address_zip"]

schema_2=StructType([
                      StructField("submitted_at",TimestampType(),True),\
                      StructField("submitted_yyyy_mm", StringType(),True),\
                      StructField("order_id",StringType(),True),\
                      StructField("customer_id",StringType(),True),\
                      StructField("sales_rep_id",StringType(),True),\
                      StructField("shipping_address_attention",StringType(),True),\
                      StructField("shipping_address_address",StringType(),True),\
                      StructField("shipping_address_city",StringType(),True),\
                      StructField("shipping_address_state",StringType(),True),\
                      StructField("shipping_address_zip",IntegerType(),True),\
                      StructField("ingest_file_name",StringType(),True),\
                      StructField("ingested_at",TimestampType(),True),\
                      ])


dataset2 = batched_orders\
.withColumn("submitted_at", to_timestamp(from_unixtime(col("submitted_at"),"yyyy-MM-dd HH:mm:ss")))\
.withColumn("shipping_address_zip", col("shipping_address_zip").cast(IntegerType()))\
.drop("sales_rep_ssn","sales_rep_first_name","sales_rep_last_name","sales_rep_address","sales_rep_city","sales_rep_state","sales_rep_zip","product_id","product_quantity","product_sold_price")\
.drop_duplicates(subset=subset_2)\
.withColumn("submitted_yyyy_mm", col("submitted_at").substr(1,7))




dataset2.write.format("delta").partitionBy("submitted_yyyy_mm").mode("overwrite").saveAsTable(orders_table)

In [0]:
dataset2.printSchema()

root
-- submitted_at: timestamp (nullable = true)
-- order_id: string (nullable = true)
-- customer_id: string (nullable = true)
-- sales_rep_id: string (nullable = true)
-- shipping_address_attention: string (nullable = true)
-- shipping_address_address: string (nullable = true)
-- shipping_address_city: string (nullable = true)
-- shipping_address_state: string (nullable = true)
-- shipping_address_zip: integer (nullable = true)
-- ingest_file_name: string (nullable = true)
-- ingested_at: timestamp (nullable = true)
-- submitted_yyyy_mm: string (nullable = true)

In [0]:
display(dataset2)

submitted_at,order_id,customer_id,sales_rep_id,shipping_address_attention,shipping_address_address,shipping_address_city,shipping_address_state,shipping_address_zip,ingest_file_name,ingested_at,submitted_yyyy_mm
2019-07-28T16:00:00.000+0000,0000cdba-66ef-4cf9-a07e-7da7da6b6dcc,1f346308-f3b5-412a-9022-c0cda208debc,92f62533-fec8-49c6-8661-f463bb6e095d,Mary Workman,942 N Wren Street,Broken Arrow,OK,74546,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2019.csv,2022-03-01T16:08:19.770+0000,2019-07
2017-07-22T13:00:00.000+0000,0003ecad-6827-4ad3-a80d-fbeb1569d2cb,00b54fb1-a8cf-471d-869b-e368a77a5f2f,4a888d8f-afe0-4ba3-b76d-08f25e0dea27,Waverly Estrada,898 Dalecroft Trail W,Rancho Cucamonga,CA,93774,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000,2017-07
2018-08-27T10:00:00.000+0000,000410cd-e8b5-4137-815b-255d7c9828cf,b79db4ce-e911-4e10-a87f-baca8869862c,761b3d8b-a96c-42fa-ba22-a0017621cbea,Alexander Graves,736 N Hampton Lane,Oceanside,CA,95979,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000,2018-08
2017-05-01T17:00:00.000+0000,0008c4b7-3eab-4805-95f3-497c7e11f27a,fb514f0e-8535-4cb6-acdb-424c732f0062,af0f3842-846d-4e3f-9763-049978937827,Davis Garner,109 Wales Plaza E,Roseville,CA,92280,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000,2017-05
2018-05-05T13:00:00.000+0000,000a2d65-9619-4cb1-8d6b-eadca7f50898,d8c566af-5998-4178-9c32-b9ad88a95428,4b2197cf-2d8a-453c-9ca9-0199c90903d7,Fernando Jordan,42 E Sierra Blanca Court,Torrance,CA,96049,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000,2018-05
2019-01-28T06:00:00.000+0000,000a765a-38fe-4e19-93e6-2dc552f84eef,f07e0e28-803a-492d-b6b3-e5382044e1ae,4898862a-68d3-43e5-8627-caf6933bdaec,Tanner Gross,456 Ravenel Court,Gainesville,FL,32936,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2019.csv,2022-03-01T16:08:19.770+0000,2019-01
2019-04-04T06:00:00.000+0000,0013c783-52d3-4b6b-b6a4-0f985c4abacd,c3faf37b-f2f8-4b95-88d4-0208a69b4ba6,4afd2259-f7e8-4d2a-8643-33d68c4b5424,Malachi Crane,298 Burbank Lane E,Coral Springs,FL,33930,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2019.csv,2022-03-01T16:08:19.770+0000,2019-04
2017-07-20T17:00:00.000+0000,0015677d-b05b-49c7-a9b0-d53abd852b49,80592e9d-b11e-487f-9565-0b7f9af10f9c,95a919b4-5c80-4242-bf0e-66e3443ee504,Saoirse Lee,395 Rockville Place,Lewisville,TX,78629,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2017.txt,2022-03-01T15:40:49.989+0000,2017-07
2019-11-05T18:00:00.000+0000,001606e2-84a0-4ba7-9576-84f62d162767,eb151412-bf17-4a44-8c99-3191d7363389,6433aa8c-b6dd-47fd-afe4-c44b2f3b46d6,Averi Haynes,311 Rebusmen Road N,Columbus,GA,31198,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2019.csv,2022-03-01T16:08:19.770+0000,2019-11
2018-04-25T16:00:00.000+0000,00168146-ed15-4d18-98b6-d0a65e418035,d1422979-3a3c-42c5-aefc-e0d124452f4c,de37844d-425e-47d0-a996-f02d070449ff,Gerald Alvarado,379 Deskin Lane,Renton,WA,99204,dbfs:/dbacademy/class+001@databricks.com/developer-foundations-capstone/raw/orders/batch/2018.csv,2022-03-01T15:42:18.844+0000,2018-04


### Reality Check #3.D
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_d()

Points,Test,Result
1,The current database is dbacademy_class_001_databricks_com_developer_foundations_capstone,
1,The table orders exists,
1,The table orders is a managed table,
1,Using the Delta file format,
1,Schema is valid,
1,"Expected 195,698 records",
1,Non-null (properly parsed) submitted_at,
1,Partitioned by submitted_yyyy_mm,
1,Found 36 partitions,


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3.E - Extract Line Items</h2>

Now that we have extracted sales reps and orders, we next want to extract the specific line items of each order.

**In this step you will need to:**
* Load the table **`batched_orders`** (identified by the variable **`batch_temp_view`**)
* Retain the following columns (see schema below)
  * The correlating ID columns: **`order_id`** and **`product_id`**
  * The two product-specific columns: **`product_quantity`** and **`product_sold_price`**
  * The two ingest columns: **`ingest_file_name`** and **`ingested_at`**
* Convert various columns from their string representation to the specified type:
  * The column **`product_quantity`** should be represented as an **`Integer`**
  * The column **`product_sold_price`** should be represented as an **`Decimal`** with two decimal places as in **`decimal(10,2)`**
* Load the dataset to the managed delta table **`line_items`** (identified by the variable **`line_items_table`**)

**Additional Requirements:**
* The schema for the **`line_items`** table must be:
  * **`order_id`**:**`string`**
  * **`product_id`**:**`string`**
  * **`product_quantity`**:**`integer`**
  * **`product_sold_price`**:**`decimal(10,2)`**
  * **`ingest_file_name`**:**`string`**
  * **`ingested_at`**:**`timestamp`**

### Implement Exercise #3.E

Implement your solution in the following cell:

In [0]:
# TODO
# Use this cell to complete your solution

from pyspark.sql.types import DecimalType

dataset_3 = batched_orders.\
drop("submitted_at","customer_id","sales_rep_id","sales_rep_ssn","sales_rep_first_name","sales_rep_last_name","sales_rep_address","sales_rep_city","sales_rep_state","sales_rep_zip","shipping_address_attention","shipping_address_address","shipping_address_city","shipping_address_state","shipping_address_zip")\
.withColumn("product_quantity", col("product_quantity").cast("Int"))\
.withColumn("product_sold_price", col("product_sold_price").cast(DecimalType(precision=10, scale=2)))

dataset_3.write.format("delta").mode("overwrite").saveAsTable(line_items_table)

### Reality Check #3.E
Run the following command to ensure that you are on track:

In [0]:
reality_check_03_e()

Points,Test,Result
1,The current database is dbacademy_class_001_databricks_com_developer_foundations_capstone,
1,The table line_items exists,
1,The table line_items is a managed table,
1,Using the Delta file format,
1,Schema is valid,
1,"Expected 1,175,870 records",


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise #3 - Final Check</h2>

Run the following command to make sure this exercise is complete:

In [0]:
reality_check_03_final()

Wrote 17 bytes.

Points,Test,Result
1,Reality Check 03.A passed,
1,Reality Check 03.B passed,
1,Reality Check 03.C passed,
1,Reality Check 03.D passed,
1,Reality Check 03.E passed,


-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>